In [ ]:
!pip install -e ../../geoai

### Reload Library

In [1]:
import geoai
import geoai.label_utils
import geoai.utils

import sys
sys.path.append("../")

import importlib
importlib.reload(geoai)
importlib.reload(geoai.label_utils)
importlib.reload(geoai.utils)

<module 'geoai.utils' from 'C:\\Users\\GRSS NATAWARA\\Documents\\TA Remote Sensing\\geoai\\geoai\\utils.py'>

## Train single image

### Build dataset [stride 64]

In [2]:
import os

image_path = "../data/raw/images/S2H_2023_2023_07_30_nodata.tif"
label_path = "../data/raw/cdl/2023_30m_cdls_10m_remap.tif"

out_name = "crop_mapping_2023_07_30"
data_folder = f"../data/processed/{out_name}"

model_folder = f"../models/{out_name}"
os.makedirs(model_folder, exist_ok=True)

# this function will create folders (annotations, images, labels)
# tiles = geoai.export_geotiff_tiles(
#     in_raster=image_path,
#     out_folder=data_folder,
#     in_class_data=label_path,
#     tile_size=256,
#     stride=64,
#     buffer_radius=0,
#     skip_empty_tiles=True,
# )

### Train DeepLabV3+ [oversampling]

#### Setup loss criterios

In [5]:
import torch
from geoai import losses

device = "cuda" if torch.cuda.is_available() else "mps"
print("Using device:", device)

criterion = losses.DECBLoss(
    num_classes=11,
    beta=0.9999,          # smoothing factor (as in paper)
    loss_type="ce",       # or "focal"
    gamma=2.0,            # focal gamma
    ignore_index=255,
    device=device
)

Using device: cuda


#### Train model

In [ ]:
geoai.train_segmentation_model(
    images_dir=f"{data_folder}/images",
    labels_dir=f"{data_folder}/labels",
    output_dir=f"{model_folder}/deeplabv3plus_efb4_models",
    architecture="deeplabv3plus",
    encoder_name="efficientnet-b4",
    encoder_weights=None,
    num_channels=9,
    num_classes=11, # background and 10 crop classes
    batch_size=8,
    num_epochs=100,
    learning_rate=0.0005,
    val_split=0.2,
    # criterion=criterion,
    verbose=True,
    class_balanced=True,
    save_best_only=False,
    checkpoint_interval=5
)

Using device: cuda
Found 9112 image files and 9112 label files
Training on 7289 images, validating on 1823 images
Checking image sizes for compatibility...
All sampled images have the same size: (256, 256)
No resizing needed.
Building class-balanced sampler (strategy='presence') ...
Class-balanced sampler ready. Non-empty classes: 11/11.
Using class balanced sampler
Testing data loader...
Data loader test passed.
Starting training with deeplabv3plus + efficientnet-b5
Model parameters: 29,497,579
Epoch: 1, Batch: 1/911, Loss: 2.4053, Time: 0.98s
Epoch: 1, Batch: 11/911, Loss: 2.1751, Time: 3.70s
Epoch: 1, Batch: 21/911, Loss: 1.6029, Time: 3.61s
Epoch: 1, Batch: 31/911, Loss: 1.5354, Time: 3.62s
Epoch: 1, Batch: 41/911, Loss: 1.5763, Time: 3.65s
Epoch: 1, Batch: 51/911, Loss: 0.9991, Time: 3.60s
Epoch: 1, Batch: 61/911, Loss: 1.1627, Time: 3.54s
Epoch: 1, Batch: 71/911, Loss: 1.4692, Time: 3.54s
Epoch: 1, Batch: 81/911, Loss: 1.9119, Time: 3.54s
Epoch: 1, Batch: 91/911, Loss: 1.4102, Ti

#### Plot model perf

In [ ]:
geoai.plot_performance_metrics(
    history_path=f"{model_folder}/deeplabv3plus_efb4_models/training_history.pth",
    figsize=(15, 5),
    verbose=True,
)

#### Run inference

In [ ]:
# Define paths
model_path = f"{model_folder}/deeplabv3plus_efb4_models/best_model.pth"
test_raster_path = "../data/raw/images/S2H_2024_2024_07_29_nodata.tif"
predicted_path = f"../data/predicted/{out_name}/S2H_2024_2024_07_29_pred.tif"

# Run semantic segmentation inference
geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=predicted_path,
    model_path=model_path,
    architecture="deeplabv3plus",
    encoder_name="efficientnet-b4",
    num_channels=9,
    num_classes=11,
    window_size=256,
    overlap=64,
    batch_size=8,
)

Input file format: GeoTIFF (.tif)
Processing 1102 windows...


1170it [00:16, 70.30it/s]                          


Predicted classes: 11 classes, Background: 47.4%
Inference completed in 26.37 seconds
Saved prediction to ../data/predicted/crop_mapping_2023_07_30/S2H_2024_2024_07_29_pred.tif


#### Remap class pred

In [ ]:
from geoai import label_utils

# Map them into compact range
mapping = {
    0: 0,   # Background
    1: 1,   # Corn
    3: 2,   # Rice
    24: 3,  # Winter Wheat
    36: 4,  # Alfalfa
    54: 5,  # Tomatoes
    69: 6,  # Grapes
    75: 7,  # Almonds
    76: 8,  # Walnuts
    210: 9, # Prunes
    211: 10 # Olives
}

# Remap back to original class codes
label_utils.remap_back_raster(
    in_path=predicted_path,
    out_path=predicted_path.replace(".tif", "_remap.tif"),
    mapping=mapping
)

Unique compact values in prediction: [ 0  1  2  3  4  5  6  7  8  9 10]
Saved restored raster to ../data/predicted/crop_mapping_2023_07_30/S2H_2024_2024_07_29_pred_remap.tif
Unique values: [  0   1   3  24  36  54  69  75  76 210 211]


#### Visualize pred

In [ ]:
import sys
sys.path.append("../")
from utils.constants import USDA_CDL_COLORS

geoai.view_raster_with_labels(
    raster_image=test_raster_path,
    label_raster=predicted_path,
    class_mapping={
        0: "Background",
        1: "Corn",
        3: "Rice",
        24: "Winter Wheat",
        36: "Alfalfa",
        54: "Tomatoes",
        69: "Grapes",
        75: "Almonds",
        76: "Walnuts",
        210: "Prunes",
        211: "Olives"
    },
    class_colors={
        0: USDA_CDL_COLORS.get(0),
        1: USDA_CDL_COLORS.get(1),
        3: USDA_CDL_COLORS.get(3),
        24: USDA_CDL_COLORS.get(24),
        36: USDA_CDL_COLORS.get(36),
        54: USDA_CDL_COLORS.get(54),
        69: USDA_CDL_COLORS.get(69),
        75: USDA_CDL_COLORS.get(75),
        76: USDA_CDL_COLORS.get(76),
        210: USDA_CDL_COLORS.get(210),
        211: USDA_CDL_COLORS.get(211)
    },
    raster_indexes=(3, 2, 1),
    raster_layer_name="Satellite",
    label_layer_name="Labels",
    legend_title="Crop Classes",
    opacity=0.5,
)

Map(center=[38.929975, -121.751141], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…